In [1]:
import pandas as pd
import json

In [2]:
import math
def make_verbose(start_with, done, total, max_lines=25):
    num_lines = int((done/total)*max_lines)
    lines = "="*num_lines
    spaces = "-"*(max_lines-num_lines)
    print(f"\r{start_with}  [{lines}>{spaces}]  {done}/{total}", end="", sep=" ", flush=True)

In [3]:
df = pd.read_csv("./outputs/combined/ias-clean-final.csv")

In [12]:
edu_df = pd.DataFrame(columns=["Name","Service","Cadre","Allotment Year","Date of Birth","Date of Joining","Source of Recruitment","Gender","Place of Domicile","Mother Tongue","Languages Known","Retirement Reason","Qualification/University/Institute","Subject","Division","Dataset name","Clean Ids"])
exp_df = pd.DataFrame(columns=["Name","Service","Cadre","Allotment Year","Date of Birth","Date of Joining","Source of Recruitment","Gender","Place of Domicile","Mother Tongue","Languages Known","Retirement Reason","Designation/Level","Ministry/Department/Office/Location","Organisation","Experience(major/minor)","Period(From/To)","Dataset name","Clean Ids"])
edu_columns = ["Qualification/University/Institute","Subject","Division"]
exp_columns = ["Designation/Level","Ministry/Department/Office/Location","Organisation","Experience(major/minor)","Period(From/To)"]

In [5]:
#create thhe json objects
def create_json_object(row):
    row_object = {}
    for key in row.keys():
        if key in list_columns:
            row_object[key] = row[key].split("|")
        else:
            row_object[key] = str(row[key]).strip()
    return row_object

In [6]:
my_json = []
length = len(df.index)
for i in df.index:
    make_verbose("", i+1, length)
    row_dict = dict(df.iloc[i])
    my_json.append(create_json_object(row_dict))

  [=========================>]  11569/11569

In [12]:
with open("./outputs/formats/ias-json.json", "w") as my_json_file:
    json.dump(my_json, my_json_file, indent=4)

In [6]:
new_df = df

In [66]:
def create_row_by_criteria(row, df_columns, required_columns, seperator="|"):
    length_of_list = len(row[required_columns[0]].split(seperator))
    generated_rows = []
    for column in required_columns:
        row[column] = str(row[column]).split(seperator)
    for i in range(length_of_list):
        new_row = {}
        for column in df_columns:
            if column in required_columns:
                to_add = row[column][i]
            else:
                to_add = row[column]
            
            new_row[column] = to_add
        
        generated_rows.append(new_row)
    
    return generated_rows

In [76]:
length = len(df.index)
for i in range(length):
    make_verbose("", i+1, length)
    row = dict(new_df.iloc[i])
    edu_rows = create_row_by_criteria(row, edu_df.columns, edu_columns)
    exp_rows = create_row_by_criteria(row, exp_df.columns, exp_columns)
    edu_df = edu_df.append(edu_rows, ignore_index=True)
    exp_df = exp_df.append(exp_rows, ignore_index=True)

  =========================>  11569/11569

In [77]:
edu_df.to_csv("./outputs/formats/ias-education.csv", index=False)
exp_df.to_csv("./outputs/formats/ias-postings.csv", index=False)